In [21]:
import websockets
import pandas as pd



In [22]:
channels = pd.read_csv("channels_by_category.csv")
n_messages = 10 # number of messages to scrape
# n_channels = len(channels)
n_channels = 5 # for testing, limit to first 5 channels
channels = channels.head(n_channels)

In [23]:
async def twitch_chat_scraper(channel, n_messages):
    uri = "wss://irc-ws.chat.twitch.tv/"
    messages = []
    async with websockets.connect(uri) as websocket:
        await websocket.send("PASS SCHMOOPIIE\r\n")
        await websocket.send("NICK justinfan26589\r\n")
        await websocket.send(f"JOIN #{channel}\r\n")
        print(f"Connected and joined #{channel}. Listening for messages...")
        while True:
            try:
                if len(messages) >= n_messages:
                    df = pd.DataFrame({'message': messages})
                    df.to_csv(f"{channel}_chat_messages.csv", index=False)
                    break
                message = await websocket.recv()
                if "PRIVMSG" in message:
                    parts = message.split("PRIVMSG", 1)
                    if len(parts) > 1:
                        msg_text = parts[1].split(":", 1)[-1].strip()
                        print(msg_text)
                        messages.append(msg_text)
            except Exception as e:
                print(f"Error: {e}")
                break


In [24]:
for channel in channels['channelId']:
    print(f"Scraping chat messages from channel: {channel}")
    await twitch_chat_scraper(channel, n_messages)

Scraping chat messages from channel: kaicenat
Connected and joined #kaicenat. Listening for messages...
You sleep great when your day is productive
🤣🤣
you would wake up and be in the shark’s stomach
SeemsGood Kreygasm ResidentSleeper WutFace BigSad NotLikeThis LUL <3 HeyGuys WutFace LUL Kreygasm NotLikeThis Kreygasm NotLikeThis Kappa Kreygasm HeyGuys BigSad VoHiYo BigSad <3 ResidentSleeper Kreygasm HeyGuys LUL WutFace Kappa ResidentSleeper BigSad NotLikeThis BigSad <3 Kreygasm Kappa HeyGuys SeemsGood LUL WutFace <3 ResidentSleeper Kreygasm HeyGuys BigSad NotLikeThis BigSad LUL ResidentSleeper Kappa Kreygasm SeemsGood
W RAKAI.... #1 RAGE BAITER #UNDEFEATED
8 hours til mafiathon over ggs
!watchtime
@KingV916 KingV916 has spent 1 month and 6 days in online chat.
team cry baby Dez 😭😭😭😭😭😭😭😭😭😭😭😭
deshae content is funny asl folks not tryna to get wet💀😂😂
Scraping chat messages from channel: cdawg
Connected and joined #cdawg. Listening for messages...
Susge The mods are in cahoots with the clan